In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from Bio import Entrez
from tqdm import tqdm
import pickle

In [2]:
data = pd.read_csv (r'data/SFARI-Genes.csv')
cols = data.columns
N, D = data.shape
print(cols)

Index(['status', 'gene-symbol', 'gene-name', 'ensembl-id', 'chromosome',
       'genetic-category', 'gene-score', 'syndromic', 'number-of-reports'],
      dtype='object')


In [3]:
def save_to_txt_file(contents, filename):
    DIR = os.getcwd() + "/results/" + filename
    
    with open(DIR, "w") as f:
        f.write(contents)
        
def save_to_pickle_file(contents, filename):
    DIR = os.getcwd() + "/data/" + filename
    
    with open(DIR, "wb") as handle:
        pickle.dump(contents, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
def open_pickle_file(filename):
    DIR = os.getcwd() + "/data/" + filename
    
    with open(DIR, "rb") as handle:
        data = pickle.load(handle)
        
    return data

# Task 1

In [4]:
gene_symbols = data['gene-symbol'].unique()
print(gene_symbols.size)

1023


In [5]:
# Save Entrez scores

In [45]:
Entrez.email = "s1803764@ed.ac.uk"
output = ""
eid = {}

for i in tqdm(range(gene_symbols.size)):
    g = gene_symbols[i]
    handle = Entrez.esearch(db="gene", retmax=10, term=g + "[sym] homo sapiens[Organism]")
    record = Entrez.read(handle)
    #output += g + ": " + ", ".join(str(x) for x in record['IdList'])
    #output += "\n"
    eid[g] = record['IdList']
    
    if len(record["IdList"]) == 0:
        print(g)
    
#save_to_txt_file(output, "task2-1-eid.txt")

 46%|██████████████████▏                     | 466/1023 [04:42<05:37,  1.65it/s]


KeyboardInterrupt: 

In [22]:
save_to_pickle_file(eid, "eid.pickle")

# Task 2

In [6]:
gene2go = pd.read_csv(os.getcwd() + "/data/gene2go", dtype=str, delimiter="\t")
save_to_pickle_file(gene2go, "gene2go.pickle")

In [20]:
gene2go

,#tax_id,GeneID,GO_ID,Evidence,Qualifier,GO_term,PubMed,Category
0,3702,814629,GO:0005634,ISM,located_in,nucleus,-,Component
1,3702,814629,GO:0008150,ND,involved_in,biological_process,-,Process
2,3702,814630,GO:0003700,ISS,enables,DNA-binding transcription factor activity,11118137,Function
3,3702,814630,GO:0005634,ISM,located_in,nucleus,-,Component
4,3702,814630,GO:0006355,TAS,acts_upstream_of_or_within,"regulation of transcription, DNA-templated",11118137,Process
...,...,...,...,...,...,...,...,...
2084173,559292,65052912,GO:0005575,ND,is_active_in,cellular_component,-,Component
2084174,559292,65052912,GO:0008150,ND,involved_in,biological_process,-,Process
2084175,559292,65052913,GO:0003674,ND,enables,molecular_function,-,Function
2084176,559292,65052913,GO:0005575,ND,is_active_in,cellular_component,-,Component


In [8]:
gene2go = open_pickle_file("gene2go.pickle")
#eid = pd.read_pickle(os.getcwd() + "/data/eid.pickle")

In [43]:
go_terms = {}

tax_ids = gene2go["#tax_id"]
human_gene_indexes = tax_ids[tax_ids == "9606"].index
human_genes = gene2go.filter(items=human_gene_indexes, axis = 0)
human_gene_ids = human_genes["GeneID"]
print(human_gene_indexes.size)

for g in tqdm(gene_symbols):
    gene_entrez = eid[g]
    
    if len(gene_entrez) == 0:
        break
    else:
        gene_entrez = gene_entrez[0]

    gene_index = human_gene_ids[human_gene_ids == gene_entrez].unique()

    if gene_index.size >= 1:
        gene_index = int(gene_index[0])

    print(gene_index)
    go_term = gene2go.iloc[gene_index]["GO_term"]
    go_terms[g] = go_term

    print(g)
    print(gene_entrez)
    print(go_term)
    print()

save_to_pickle_file(go_terms, "go-terms.pickle")

331423


  0%|▏                                         | 5/1023 [00:00<00:22, 45.82it/s]

18
ABAT
18
mitochondrial electron transport, ubiquinol to cytochrome c

10349
ABCA10
10349
protein binding

154664
ABCA13
154664
DNA-7-methylguanine glycosylase activity

10347
ABCA7
10347
inward rectifier potassium channel activity

1636
ACE
1636
auxin-activated signaling pathway

43
ACHE
43
intracellular anatomical structure

60
ACTB
60
plastoglobule

51412
ACTL6B
51412
cytosol

81
ACTN4
81
L-alanine transmembrane transporter activity



  1%|▌                                        | 15/1023 [00:00<00:22, 44.80it/s]

95
ACY1
95
phospholipid metabolic process

100
ADA
100
response to UV-B

109
ADCY3
109
response to auxin

111
ADCY5
111
protein serine/threonine kinase activity

132
ADK
132
biosynthetic process

23394
ADNP
23394
protein binding

140
ADORA3
140
mRNA binding

158
ADSL
158
extracellular space

2334
AFF2
2334
cytosol

116987
AGAP1
116987
transmitting tissue development



...

 50%|████████████████████                    | 513/1023 [00:11<00:11, 45.13it/s]

4173
MCM4
4173
cytosol

4175
MCM6
4175
plasma membrane

79648
MCPH1
79648
regulation of protein localization to cell surface

161357
MDGA2
161357
translation

4204
MECP2
4204
chloroplast

3827
KNG1
3827
peptidyl-prolyl cis-trans isomerase activity

81887
LAS1L
81887
mitochondrion

8861
LDB1
8861
monooxygenase activity

3949
LDLR
3949
carbohydrate transmembrane transporter activity

4010
LMX1B
4010
dihydrofolate reductase activity



 51%|████████████████████▍                   | 524/1023 [00:11<00:11, 45.22it/s]

57689
LRRC4C
57689
protein binding

84445
LZTS2
84445
cytoplasm

116931
MED12L
116931
nucleus

9969
MED13
9969
nucleus

23389
MED13L
23389
red light signaling pathway

4208
MEF2C
4208
cytosol

55857
KIZ
55857
Golgi apparatus

4137
MAPT
4137
dolichol biosynthetic process

100128977


IndexError: single positional indexer is out-of-bounds

In [39]:
human_gene_indexes[0]

643502

In [44]:
eid[""]

[]